In [3]:
import os
import csv
import time
import torch
import requests
from datasets import load_dataset
from google.cloud import translate_v2 as translate
from concurrent.futures import ThreadPoolExecutor    # Concurrent execution using threads
gcp_project_id = "artful-tractor-338209"   # Set the Google Cloud Project ID

In [58]:
possible_configs = [
    # "main",
    # "socratic",
    # 'ARC-Challenge',
    # 'ARC-Easy',
    'bbc hindi nli'
]
dataset = []
for config in possible_configs:
    dataset_slice = load_dataset("bbc_hindi_nli", config,ignore_verifications=True)
    dataset.append(dataset_slice)

Found cached dataset bbc_hindi_nli (C:/Users/karti/.cache/huggingface/datasets/bbc_hindi_nli/bbc hindi nli/1.1.0/cd576e2277553122ee57670998e5122a0699a0153a4ccc21dd554e424bd17e58)


  0%|          | 0/3 [00:00<?, ?it/s]

In [59]:
# dataset
for g in dataset:
    print(g['train'])
    print(g['test'])
    print(g['validation'])

Dataset({
    features: ['premise', 'hypothesis', 'label', 'topic'],
    num_rows: 15552
})
Dataset({
    features: ['premise', 'hypothesis', 'label', 'topic'],
    num_rows: 2592
})
Dataset({
    features: ['premise', 'hypothesis', 'label', 'topic'],
    num_rows: 2580
})


In [ ]:
access_token = 'ya29.a0AfB_byDPJReEjznltcD1LepRfvKCPFBx_eFOm2BPwVq6z9Agn16pFhR7x7UMKRYQSAf01y9Gwk8vPmHVE0lpWtGpF_Kq2wcqeFtnZDOU5jilu-rJsZA0Vra2CvUzmu2XdmYNNz1eRGV72buKBonuQQMwuolwW4y5dqBzaCgYKAdoSARASFQHGX2MiN66GagSurFyiLy4yrIErHQ0171'

## Get access token for romanization

1. go to link https://developers.google.com/oauthplayground/ 
2. select Cloud Translation API v3, under it select both https://www.googleapis.com/auth/cloud-platform
https://www.googleapis.com/auth/cloud-translation
3. authorize
4. exchange authorization code for tokens and copy the access token.

   #### valid for one hour only

In [55]:
def romanize( text: str):
    url = 'https://translation.googleapis.com/v3/projects/artful-tractor-338209/locations/global:romanizeText'

    headers = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    }
    data = {
    'contents': [text],
    'sourceLanguageCode': 'hi',
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        # print(result)
        romanized_text = result['romanizations'][0]['romanizedText']
        return f'{romanized_text}'
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [64]:
romanize("कार्तिक आप कैसे हैं?")

'kaartik aap kaise hain?'

In [56]:
%%time

romanize("{'text': ['सूखे हुए हाथ', 'गीले हुए हाथ', 'तेल से ढके हुए हाथ', 'लोशन से ढके हुए हाथ'], 'label': ['ए', 'बी', 'सी', 'डी']}")

CPU times: total: 0 ns
Wall time: 879 ms


'{taixt: [sookhe hue haath, geele hue haath, tel se dhake hue haath, loshan se dhake hue haath], labail: [e, bee, see, dee]}'

In [57]:
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

Number of CPU cores: 16


In [62]:
%%time

for i in range(len(possible_configs)):
    for set in dataset[i]:
        set_list = []
        columns = ['premise','hypothesis']
        
        for col in columns:
            values = [str(item[col]) for item in dataset[i][set]]
            # print(values[1:2])
            
            # Use ThreadPoolExecutor for parallel translation
            if __name__ == '__main__':
                result =[]
                with ThreadPoolExecutor(max_workers=16) as exe:
                    # Maps the method with a list of values.
                    result = list(exe.map(romanize,values[10:12]))
                    
                    # batch_size = 100
                    # for j in range(0,len(values),batch_size):
                    #     j_end = min(j + batch_size, len(values))
                    #     res = list(exe.map(translate_text,values[j:j_end]))
                    #     time.sleep(2)
                    #     result.extend(res)
                    
            set_list.append(result)

        # Create folders for each configuration
        current_directory = os.getcwd()
        
        # Specify the path of the 'config' folder
        config_folder_path = os.path.join(current_directory, possible_configs[i])
        
        # Create the 'config' folder
        os.makedirs(config_folder_path, exist_ok=True)       
    
        # Transpose the 2D list
        transposed_data = list(map(list, zip(*set_list)))
        path = os.path.join(possible_configs[i], f'{set}.csv')

        # append to previosly created csv file in case full dataset was not converted
        with open(path, 'a', encoding='utf-8') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            # write.writerow(columns)
            write.writerows(transposed_data)

CPU times: total: 1.11 s
Wall time: 6.6 s


In [63]:
print(result)

['yah vigyaan kee soochana hai|', 'yah vigyaan kee soochana nahin hai|']
